In [1]:
##################################################################################################
# Patch-based Classification of Breast Cancer Histology Images using CNNs
# LE48: MiniProject
# Jan Ondras (jo356), Trinity College
# 2017/2018
##################################################################################################
# Train CNN model                        (x type)
##################################################################################

#import matplotlib.pyplot as plt
import numpy as np
# import cv2
import glob
import os
from time import time
from math import ceil

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import metrics
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.utils import multi_gpu_model
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
import keras

seed = 7 #7
np.random.seed(seed)

# seeds = [7, 34, 888, 456456, 33, 16947, 7764, 3396, 20500, 3009467] # 10 random seeds = 10 rand restarts

# for seed_index, seed in enumerate(seeds): # 10 random restarts
#     np.random.seed(seed)
#     if seed == 7 or seed == 34:        # TO DELETE
#         continue

###################################################################################### TO SET
overwrite = True
overwrite = False

# fixed_augment = True
fixed_augment = False
fixedAndRand_augment = False
# fixedAndRand_augment = True

N_patch_augment = 8

target_size = (256,256)
model_ID = 'x00' # + str(seed_index)
patch_width = 256                     # 256, 512, 1024
patch_height = patch_width
patch_stride = 128                    # 256, 512
model_type = 'SN_' + str(patch_width) + '_' + str(patch_stride) + '_' + model_ID

RGB_means = [
    180.375933345 , 148.276127037 , 174.592562423
] 
# AWS: 180.375933345 , 148.276127037 , 174.592562423
# HPC: 180.7795009644826 , 148.3537065177495 , 174.59451631969864
######################################################################################
# Create model directory for model checkpointing
if overwrite == False:
    if not os.path.exists('./../Model/' + model_type):
        os.mkdir('./../Model/' + model_type)
    else:
        raise NameError('Model directory ' + './../Model/' + model_type + ' already exists!')
model_checkpoint_path_prefix = './../Model/' + model_type + '/' + model_type + '_'
print model_type

# History file for checkpointing loss, acc and time per epoch
history_filename = './../History/history_' + model_type + '.txt'
if overwrite == False:
    if os.path.exists(history_filename):
        raise NameError('History file ' + history_filename + ' already exists!')

classes = ['Normal', 'Benign', 'InSitu', 'Invasive'] # correspond to labels 0,1,2,3 in this order

# Image size (assume same for all images)
img_width =  2048
img_height = 1536
pix_scale = 0.42 # micrometers

# Number of examples (whole images) per class before data augmentation
# N_imgs_per_class = 100

N_patches_x = (((img_width - patch_width) / patch_stride) + 1)
N_patches_y = (((img_height - patch_height) / patch_stride) + 1)
N_patches_per_img = N_patches_x * N_patches_y
print "Number of patches per image: ", N_patches_per_img

path_prefix = './../Dataset/ICIAR2018_BACH_Challenge/Photos_SN/' 
train_valid_split = 0.75
if fixed_augment:
    train_data_dir = path_prefix+'aug_patches_' + str(patch_width) + '_' + str(patch_stride) + '_' + str(N_patch_augment) + '/train'
else:
    train_data_dir = path_prefix+'patches_' + str(patch_width) + '_' + str(patch_stride) + '/train'
validation_data_dir = path_prefix+'patches_' + str(patch_width) + '_' + str(patch_stride) + '/validation'

# N_train_samples = train_valid_split * N_imgs_per_class * len(classes) * N_patches_per_img
# N_validation_samples = (1.-train_valid_split) * N_imgs_per_class * len(classes) * N_patches_per_img
if fixed_augment:
    N_train_samples = len(glob.glob(path_prefix+'aug_patches_' + str(patch_width) + '_' + str(patch_stride) + '_' + str(N_patch_augment) + '/train/*/*.tif'))
    N_validation_samples = len(glob.glob(path_prefix+'patches_' + str(patch_width) + '_' + str(patch_stride) + '/validation/*/*.tif'))
    print N_train_samples, N_validation_samples, " = ", N_patch_augment*N_patches_per_img*300, N_patches_per_img*100
else:
    N_train_samples = len(glob.glob(path_prefix+'patches_' + str(patch_width) + '_' + str(patch_stride) + '/train/*/*.tif'))
    N_validation_samples = len(glob.glob(path_prefix+'patches_' + str(patch_width) + '_' + str(patch_stride) + '/validation/*/*.tif'))
    print N_train_samples, N_validation_samples, " = ", N_patches_per_img*300, N_patches_per_img*100

epochs = 1000
train_batch_size = 256    #32
evaluate_batch_size = 256 #64 #N_validation_samples
steps_per_epoch =  int(ceil(float(N_train_samples) / train_batch_size))
validation_steps = int(ceil(float(N_validation_samples) / evaluate_batch_size))
print "Batch size: ", train_batch_size
print "Evaluate batch size: ", evaluate_batch_size
print "Steps per epoch: ", steps_per_epoch, N_train_samples // train_batch_size
print "Validation steps per epoch: ", validation_steps, N_validation_samples // evaluate_batch_size

if K.image_data_format() == 'channels_first':
    input_shape = (3, target_size[0], target_size[1])
    print "Channels first"
else:
    input_shape = (target_size[0], target_size[1], 3) # this one
    print "Channels last"


##################################################################################
def substract_mean(patch):
    # RGB
    patch[:,:,0] -= RGB_means[0]
    patch[:,:,1] -= RGB_means[1]
    patch[:,:,2] -= RGB_means[2]    
    return patch

##################################################################################

#def create_model():
# add dropout: Dropout(0.5) after dense
# strides for pooling layer = pool size

# TODO: kernel_initializer='he_uniform'

# MODEL WITH BATCH NORMALISATION

# inp = Input(shape=input_shape)
# inpN = BatchNormalization()(inp)

# c1 = Conv2D(16, (3, 3), strides=(1, 1), padding='valid', activation='relu')(inpN)
# c1 = BatchNormalization()(c1)
# p1 = MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')(c1)
# p1 = Dropout(0.1)(p1)

# c2 = Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')(p1)
# c2 = BatchNormalization()(c2)
# p2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(c2)
# p2 = Dropout(0.1)(p2)

# c3 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(p2)
# c3 = BatchNormalization()(c3)
# p3 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(c3)
# p3 = Dropout(0.1)(p3)

# c4 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(p3)
# c4 = BatchNormalization()(c4)
# p4 = MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')(c4)
# p4 = Dropout(0.1)(p4)

# c5 = Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')(p4)
# c5 = BatchNormalization()(c5)
# p5 = MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')(c5)
# p5 = Dropout(0.1)(p5)

# f1 = Flatten()(p5)

# d1 = Dense(256, activation='relu')(f1)
# d1 = BatchNormalization()(d1)
# d1 = Dropout(0.5)(d1)

# d2 = Dense(128, activation='relu')(d1)
# d2 = BatchNormalization()(d2)
# d2 = Dropout(0.5)(d2)

# out = Dense(4, activation='softmax')(d2)

# model = Model(input=inp, output=out)



model = Sequential([
    
    Conv2D(16, (3, 3), strides=(1, 1), padding='valid', input_shape=input_shape, activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
#         Dropout(0.1),

    Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'),
#         Dropout(0.1),

    Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'),
#         Dropout(0.1),

    Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
#         Dropout(0.1),

    Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
#         Dropout(0.1),
    
    # Below is with He init
#     Conv2D(16, (3, 3), strides=(1, 1), padding='valid', input_shape=input_shape, activation='relu', kernel_initializer='he_uniform'),
#     MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
# #     Dropout(0.1),

#     Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_uniform'),
#     MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'),
# #     Dropout(0.1),

#     Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='he_uniform'),
#     MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'),
# #     Dropout(0.1),

#     Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='he_uniform'),
#     MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
# #     Dropout(0.1),

#     Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_uniform'),
#     MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid'),
# #     Dropout(0.1),

    Flatten(),
#     Dense(256, activation='relu', kernel_initializer='he_uniform'),
    Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu', kernel_initializer='he_uniform'),
    Dense(128, activation='relu'),
#     Dropout(0.5),
    Dense(4, activation='softmax')
])

print (model.summary())

# Run on 8 GPUs
# model = multi_gpu_model(model, gpus=4)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(), # 'adadelta' rmsprop adam
              metrics=['accuracy'])

# For fixed augmentation
if fixed_augment:
    # Both
    if fixedAndRand_augment:
        train_datagen = ImageDataGenerator(
            preprocessing_function=substract_mean, 
            rotation_range=45, 
            fill_mode='reflect' # reflect / wrap / nearest
        ) 
    # Fixed augment only
    else:
        train_datagen = ImageDataGenerator(
            preprocessing_function=substract_mean, 
        ) 
# Random augmentation only
else:
    train_datagen = ImageDataGenerator(
        preprocessing_function=substract_mean, 
        rotation_range=180, 
        horizontal_flip=True, 
        fill_mode='reflect' # reflect / wrap / nearest
            #rotation_range=45, 
            #rescale=1./255,
            #horizontal_flip=True,
            #vertical_flip=True,
            #fill_mode='reflect'
    ) 

# test_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(
    preprocessing_function=substract_mean
)

# Data generator takes care of one hot encoding # one_hot_labels = keras.utils.to_categorical(labels, num_classes=4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    batch_size=train_batch_size,
    class_mode='categorical',
    classes=classes)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=target_size,
    batch_size=evaluate_batch_size,
    class_mode='categorical',
    classes=classes)

print "Steps per epoch:", len(train_generator), len(validation_generator)
if len(train_generator) != steps_per_epoch:
    raise ValueError('Check train steps per epoch!')
if len(validation_generator) != validation_steps:
    raise ValueError('Check valid steps per epoch!')

# Checkpoint model weights and the model itself
model_checkpoint_name = 'w-{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + model_checkpoint_name, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# Checkpoint loss and acc for val and train after every epoch
class ModelHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_loss = []
        self.train_acc = []
        self.val_loss = []
        self.val_acc = []
        self.times = []
        with open(history_filename, 'w') as f:
            f.write("")

    def on_epoch_begin(self, epoch, logs={}):
        self.starttime=time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time()-self.starttime)
        self.train_loss.append(logs.get('loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_loss.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))
        with open(history_filename, 'a') as f:
            f.write( str(logs.get('loss')) + "," + str(logs.get('acc')) + "," 
                    + str(logs.get('val_loss')) + "," + str(logs.get('val_acc')) + "," + str(self.times[-1]) + "\n")

model_hist = ModelHistory()

early_stop = EarlyStopping(monitor='val_acc', patience=10, verbose=1) 

history = model.fit_generator(
    train_generator, 
    steps_per_epoch = steps_per_epoch, 
    #steps_per_epoch=N_train_samples // train_batch_size,             # wrong!
    epochs=epochs,
    validation_data=validation_generator, 
    validation_steps = validation_steps, 
    #validation_steps=N_validation_samples // evaluate_batch_size,    # wrong!
    verbose=1,
    shuffle=True, 
    callbacks=[model_checkpoint, model_hist, early_stop]
    #use_multiprocessing=True,
    #workers=4
    )

Using TensorFlow backend.


SN_256_128_x00
Number of patches per image:  165
49500 16500  =  49500 16500
Batch size:  256
Evaluate batch size:  256
Steps per epoch:  194 193
Validation steps per epoch:  65 64
Channels last
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 84, 84, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 82, 82, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 41, 41, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 41, 64)        18496     
_________________________________________________________________
max_pooling2d

In [ ]:
#########################
# OLD, not used

##############################################################################
# Plot training and validation loss and accuracy curves
# Plot training times per epoch
##############################################################################
from matplotlib import pyplot as plt

x = range(1, len(model_hist.times)+1)

# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(x, model_hist.train_loss,'ro-')#,linewidth=2.0)
plt.plot(x, model_hist.val_loss,'bo-')#,linewidth=2.0)
plt.legend(['Training loss', 'Validation Loss'])#,fontsize=18)
plt.xticks(x, x)
plt.xlabel('Epoch')#,fontsize=16)
plt.ylabel('Loss')#,fontsize=16)
# plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(x, model_hist.train_acc,'ro-')#,linewidth=2.0)
plt.plot(x, model_hist.val_acc,'bo-')#,linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'])#,fontsize=18)
plt.xticks(x, x)
plt.xlabel('Epoch')#,fontsize=16)
plt.ylabel('Accuracy')#,fontsize=16)
# plt.title('Accuracy Curves',fontsize=16)
plt.show()

# Training time
plt.figure(figsize=[8,6])
plt.plot(x, np.array(model_hist.times)/60., 'o-')
plt.xticks(x, x)
plt.xlabel('Epoch')
plt.ylabel('Training time per epoch (min)')
plt.legend(['Mean = '+str(np.mean(model_hist.times)/60.)+" min"])
plt.show()

# Oly if whole training completes ...
# # Loss Curves
# plt.figure(figsize=[8,6])
# plt.plot(history.history['loss'],'r',linewidth=2.0)
# plt.plot(history.history['val_loss'],'b',linewidth=2.0)
# plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Loss',fontsize=16)
# plt.title('Loss Curves',fontsize=16)
# # Accuracy Curves
# plt.figure(figsize=[8,6])
# plt.plot(history.history['acc'],'r',linewidth=2.0)
# plt.plot(history.history['val_acc'],'b',linewidth=2.0)
# plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Accuracy',fontsize=16)
# plt.title('Accuracy Curves',fontsize=16)
# plt.show()

In [ ]:

    # just to check
    # print train_generator.class_indices, train_generator.classes
    # print validation_generator.class_indices, validation_generator.classes

    # early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    # model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

    # Reduce learning rate when val_loss stopped improving
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
    #                               patience=5, min_lr=0.001)
    # model.fit(X_train, Y_train, callbacks=[reduce_lr])
    # or: keras.callbacks.LearningRateScheduler(schedule)

# Custom generator
# https://www.kaggle.com/sinkie/keras-data-augmentation-with-multiple-inputs
# def generate_arrays_from_file(path):
#     while 1:
#         f = open(path)
#         for line in f:
#             # create numpy arrays of input data
#             # and labels, from each line in the file
#             x1, x2, y = process_line(line)
#             yield ({'input_1': x1, 'input_2': x2}, {'output': y})
#         f.close()

# model.fit_generator(generate_arrays_from_file('/my_file.txt'),
#                     steps_per_epoch=10000, epochs=10)